### Read in data from the sql database:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import sqlite3
%matplotlib inline

In [2]:
conn = sqlite3.connect("data/podcast.sqlite")

In [3]:
#define a function to check out the data tables in the database
def table_columns(conn, table_name):
    """Obtains variable names in the data table.
       Parameters: (1) conn is the connection to the database 
                   (2) table_name is a string containing the table name of interest
       Returns: column names
    """
    cur = conn.cursor()
    sql = "SELECT * FROM %s WHERE 1=0;" % table_name
    cur.execute(sql)
    #cursor.description is column description
    return [d[0] for d in cur.description]

In [4]:
#define a function to retrieve data from the table
def get_table(conn, table_name):
    """Obtains all the observations from the data table.
       Parameters: (1) conn is the connection to the database 
                   (2) table_name is a string containing the table name of interest
       Returns: a dataframe 
    """
    df = pd.read_sql("SELECT * FROM %s" % table_name, conn)
    return df

In [5]:
table_columns(conn, "reviews")

['podcast_id', 'title', 'content', 'rating', 'created_at']

In [6]:
reviews = get_table(conn, "reviews")
reviews.rename(columns={"title":"review_title"}, inplace=True)

In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162840 entries, 0 to 1162839
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   podcast_id    1162840 non-null  object
 1   review_title  1162840 non-null  object
 2   content       1162840 non-null  object
 3   rating        1162840 non-null  int64 
 4   created_at    1162840 non-null  object
dtypes: int64(1), object(4)
memory usage: 44.4+ MB


In [8]:
#spot check reviews, there are duplicates
mask = (reviews.podcast_id == "bad6c91efdbee814db985c7a65199604") & (reviews.review_title == "Wow in the world")
reviews[mask]

,podcast_id,review_title,content,rating,created_at
265492,bad6c91efdbee814db985c7a65199604,Wow in the world,Wow in the world is so much fun to listen to!!!,5,2019-07-03T04:38:23-07:00
265546,bad6c91efdbee814db985c7a65199604,Wow in the world,I loooooooooooooooooooooooooooove you’re podca...,5,2019-06-24T07:03:22-07:00
265564,bad6c91efdbee814db985c7a65199604,Wow in the world,WoW it is my fav podcast 😄🤪🤩,5,2019-06-23T18:23:48-07:00
265605,bad6c91efdbee814db985c7a65199604,Wow in the world,I think wow in the world is so fun and so cre...,5,2019-06-09T06:53:58-07:00
265634,bad6c91efdbee814db985c7a65199604,Wow in the world,"I love this show, I cannot stop listening to t...",5,2019-05-29T05:50:40-07:00
...,...,...,...,...,...
1134553,bad6c91efdbee814db985c7a65199604,Wow in the world,WOW IN THE WORLD IS THE BEST PODCAST IN THE WO...,5,2020-12-19T23:34:08-07:00
1136000,bad6c91efdbee814db985c7a65199604,Wow in the world,I am so glad y’all r making new episodes but m...,5,2020-12-22T19:59:10-07:00
1159175,bad6c91efdbee814db985c7a65199604,Wow in the world,OMG this is the best podcast ever!!!!🙂 I love it.,5,2021-01-29T16:46:49-07:00
1159270,bad6c91efdbee814db985c7a65199604,Wow in the world,Soooooooooo awesome!!!!!! I wished you could m...,5,2021-01-29T20:46:41-07:00


In [9]:
#check number of duplicates in the reviews and drop the duplicates
print(reviews.duplicated().sum())
reviews = reviews.drop_duplicates()

521


In [10]:
reviews.shape

(1162319, 5)

In [11]:
#about 90% of the reviews have rating >= 4...imbalance
reviews["rating"].value_counts("rating")

5    0.867504
1    0.052833
4    0.032797
3    0.024898
2    0.021968
Name: rating, dtype: float64

### Load categories and podcasts tables and merge with reviews:

In [12]:
table_columns(conn, "categories")

['podcast_id', 'category']

In [13]:
categories = get_table(conn, "categories")
categories.head()

,podcast_id,category
0,c61aa81c9b929a66f0c1db6cbe5d8548,arts
1,c61aa81c9b929a66f0c1db6cbe5d8548,arts-performing-arts
2,c61aa81c9b929a66f0c1db6cbe5d8548,music
3,ad4f2bf69c72b8db75978423c25f379e,arts
4,ad4f2bf69c72b8db75978423c25f379e,arts-design


In [14]:
#some podcasts could belong to multiple categories
#going to keep just the first one to avoid duplicating reviews when merge tables 
categories.podcast_id.value_counts().head()
categories.drop_duplicates(subset=["podcast_id"], inplace=True)

In [15]:
categories.shape

(46665, 2)

In [16]:
table_columns(conn, "podcasts")

['podcast_id', 'itunes_id', 'slug', 'itunes_url', 'title']

In [17]:
podcasts = get_table(conn, "podcasts")
podcasts.rename(columns={"title":"podcast_title"}, inplace=True)

In [18]:
podcasts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46665 entries, 0 to 46664
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   podcast_id     46665 non-null  object
 1   itunes_id      46665 non-null  int64 
 2   slug           46665 non-null  object
 3   itunes_url     46665 non-null  object
 4   podcast_title  46665 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [19]:
from functools import reduce
dfs = [reviews, categories, podcasts]
df_master = reduce(lambda left,right: pd.merge(left,right,on="podcast_id"), dfs)

In [20]:
#was expecting to have 1,162,319 entries, found some podcast_id in reviews don't exist in the other tables
df_master.shape

(895376, 10)

### Get a random sample of ~100k from the dataset for text analysis:

In [21]:
import random
rs = np.random.RandomState(11)
sample = rs.choice(df_master.shape[0], size=int(df_master.shape[0] * 0.12), replace=False)

df = df_master.iloc[sample] 

In [22]:
print(df.shape)
#checking review distribution by rating and compare to original review
df["rating"].value_counts("rating")

(107445, 10)


5    0.879976
1    0.046768
4    0.030779
3    0.022356
2    0.020122
Name: rating, dtype: float64

In [23]:
df.columns

Index(['podcast_id', 'review_title', 'content', 'rating', 'created_at',
       'category', 'itunes_id', 'slug', 'itunes_url', 'podcast_title'],
      dtype='object')

In [24]:
#number of different podcasts in the subset, slightly different numbers
len(df["podcast_id"].unique())
#len(df["podcast_title"].unique())
#len(df["slug"].unique())

19830

In [25]:
df["podcast_title"].value_counts().head(10)

My Favorite Murder with Karen Kilgariff and Georgia Hardstark    865
Wow in the World                                                 737
True Crime Obsessed                                              644
Story Pirates                                                    519
The Joe Rogan Experience                                         449
Wine & Crime                                                     439
Crime in Sports                                                  437
Armchair Expert with Dax Shepard                                 424
Zane and Heath: Unfiltered                                       388
K-Pop Daebak w/ Eric Nam                                         319
Name: podcast_title, dtype: int64

In [26]:
print(df["podcast_title"].value_counts().tail())

It Came from the Longbox          1
FilmScene                         1
Shenanigans with Rick Welbanks    1
Atheist Vanguard                  1
Movietown Movie Club              1
Name: podcast_title, dtype: int64


In [27]:
#there are 26 categories for all the podcasts
print(len(df.category.unique()))

#the most/least popular categories in terms of number of reviews
df["category"].value_counts()

26


comedy                               25889
society-culture                      19346
business                             14488
arts                                 10805
education                             8414
tv-film                               7241
christianity                          6145
kids-family                           4516
music                                 3653
religion-spirituality                 2600
technology                            2507
business-careers                       253
society-culture-personal-journals      175
society-culture-places-travel          175
spirituality                           168
business-investing                     147
judaism                                134
arts-food                              124
society-culture-philosophy             117
arts-performing-arts                   109
arts-visual-arts                       107
buddhism                               107
islam                                   96
arts-design

In [28]:
# #review ranging from 2005 to 2021
# df.sort_values("created_at").head(3)
# df.sort_values("created_at").tail(3)

### Are there podcast I known of?

In [29]:
#Invisibilia and guess what, I found Avatar The Last Airbender :D
# Hidden Brain is not in the data set...hmmm...
for title in df.podcast_title:
    if "Brain" in title:
        print(title)

The Overwhelmed Brain
Teaching Your Brain to Knit
Kwik Brain with Jim Kwik
BrainBox
Kwik Brain with Jim Kwik
The Overwhelmed Brain
Kwik Brain with Jim Kwik
Brain Burps About Books
Kwik Brain with Jim Kwik
Kwik Brain with Jim Kwik
Beauty, Brains & Business
After Dinner Investor | On The Hunt For No-Brainer Stock Investments
Kwik Brain with Jim Kwik
Brain Burps About Books
The Overwhelmed Brain
The Overwhelmed Brain
The Overwhelmed Brain
The Overwhelmed Brain
Brain Bitez
The Overwhelmed Brain
The Overwhelmed Brain
Brain Burps About Books
Kwik Brain with Jim Kwik
Kwik Brain with Jim Kwik
Kwik Brain with Jim Kwik
The Overwhelmed Brain
BrainWaves: A Neurology Podcast
Kwik Brain with Jim Kwik
Kwik Brain with Jim Kwik
Beauty, Brains & Business
The Overwhelmed Brain
BrainStuff
BrainStuff
BrainStuff
Kwik Brain with Jim Kwik
Hot Brain
Brain Bitez
Kwik Brain with Jim Kwik
Brain Dead Radio
Teaching Your Brain to Knit
Kwik Brain with Jim Kwik
Kwik Brain with Jim Kwik
Brain Bitez
Basic Brainheart
Br

In [30]:
mask = df["podcast_title"] == "Invisibilia" #"Podcast – Avatar The Last Airbender"
df[mask].sort_values("created_at").head(3)

,podcast_id,review_title,content,rating,created_at,category,itunes_id,slug,itunes_url,podcast_title
372782,a5416c179e94a74e61827c88e405b083,Great!,"I love the show, but there are some problems w...",5,2018-04-13T07:52:05-07:00,society-culture,953290300,invisibilia,https://podcasts.apple.com/us/podcast/invisibi...,Invisibilia
372767,a5416c179e94a74e61827c88e405b083,Wonderfully crafted and Beautifully Executed,"This show is funny, important, emotional, very...",5,2018-04-16T18:26:03-07:00,society-culture,953290300,invisibilia,https://podcasts.apple.com/us/podcast/invisibi...,Invisibilia
372762,a5416c179e94a74e61827c88e405b083,Amazing,An very insightful and thought provoking podca...,5,2018-04-18T13:09:29-07:00,society-culture,953290300,invisibilia,https://podcasts.apple.com/us/podcast/invisibi...,Invisibilia


### Drop columns and save df:

In [31]:
df_corpus = df.copy(deep=True)

In [32]:
df_corpus.drop(["itunes_id", "slug", "itunes_url"], axis=1, inplace=True)

In [33]:
df_corpus.head(3)

,podcast_id,review_title,content,rating,created_at,category,podcast_title
873185,a9bdaba5449189a4587793e36ce4f704,A True Crime Haiku for you,"We listen closely,\nCases of Murder and Loss\n...",5,2019-10-10T15:19:20-07:00,society-culture,Going West: True Crime
694914,c699968fd592a60c3ba7333d530907d7,Engaging in my 60's,"Love Adam & Brooke! So true, the generations c...",5,2018-12-31T04:22:13-07:00,comedy,"Thirty, Flirty, & Dying"
759771,dcfb90baae108f9938fd3a762b810cc3,Highly recommend!,Holly and her guests share inspiring and actio...,5,2017-09-11T07:05:53-07:00,business,Nine To Thrive HR


In [34]:
import pickle
df_corpus.to_pickle("data/df_100k_corpus.pkl")

### Review text cleaning (1) apply clean_text function (2) lemmatization:

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re
import time
import nltk

In [ ]:
#nltk.download()

In [36]:
df = pd.read_pickle("data/df_100k_corpus.pkl")
df.shape

(107445, 7)

In [37]:
#define function to convert all text to lower case, remove punctuation, remove words with number
def clean_text(text):
    """Cleans text by making text lowercase, removing words containing numbers, and extracting only words.
       Parameters: strings of text
       Returns: cleaned text
    """
    text = text.lower()
    #replace "\n" with a space
    text = re.sub("\n", " ", text)
    #replace words with number in them with nothing
    text = re.sub("\w*\d\w*", "", text)
    #replace anything other than words and spaces with nothing
    text = re.sub("[^a-z\s]", "", text)
    
    return text    

In [38]:
df.head(3)

,podcast_id,review_title,content,rating,created_at,category,podcast_title
873185,a9bdaba5449189a4587793e36ce4f704,A True Crime Haiku for you,"We listen closely,\nCases of Murder and Loss\n...",5,2019-10-10T15:19:20-07:00,society-culture,Going West: True Crime
694914,c699968fd592a60c3ba7333d530907d7,Engaging in my 60's,"Love Adam & Brooke! So true, the generations c...",5,2018-12-31T04:22:13-07:00,comedy,"Thirty, Flirty, & Dying"
759771,dcfb90baae108f9938fd3a762b810cc3,Highly recommend!,Holly and her guests share inspiring and actio...,5,2017-09-11T07:05:53-07:00,business,Nine To Thrive HR


In [39]:
#clean review content and add it to a new column in the data frame
df["cont_clean"] = df["content"].apply(clean_text)

### Lemmatization:

In [40]:
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [41]:
#tokenize the cleaned review content, apply lemmatization, and join the words back together
df["cont_clean"] = df["cont_clean"].apply(word_tokenize)

df["cont_clean"] = df["cont_clean"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
df["cont_clean"] = df["cont_clean"].apply(lambda x: " ".join(x))
df.head(3)

,podcast_id,review_title,content,rating,created_at,category,podcast_title,cont_clean
873185,a9bdaba5449189a4587793e36ce4f704,A True Crime Haiku for you,"We listen closely,\nCases of Murder and Loss\n...",5,2019-10-10T15:19:20-07:00,society-culture,Going West: True Crime,we listen closely case of murder and loss goin...
694914,c699968fd592a60c3ba7333d530907d7,Engaging in my 60's,"Love Adam & Brooke! So true, the generations c...",5,2018-12-31T04:22:13-07:00,comedy,"Thirty, Flirty, & Dying",love adam brooke so true the generation change...
759771,dcfb90baae108f9938fd3a762b810cc3,Highly recommend!,Holly and her guests share inspiring and actio...,5,2017-09-11T07:05:53-07:00,business,Nine To Thrive HR,holly and her guest share inspiring and action...


In [42]:
#save dataframe
df.to_pickle("data/df_100k_clean.pkl")

### View review content before and after cleaning:

In [43]:
#review samples before cleaning
for i in df["content"][:3]:
    print("Review before cleaning:", i, "\n")

Review before cleaning: We listen closely,
Cases of Murder and Loss
Going West : True Crime

Stella
Portland, Oregon 

Review before cleaning: Love Adam & Brooke! So true, the generations change, but they do recycle. I love the banter and found the discussion of bell bottoms & record players funny. It brought back memories of my own discussion with my mom as a little girl. I told her she should wear bell bottoms because they're so cool and she said I've been there, done that, and no! I get it now. 😂 

Review before cleaning: Holly and her guests share inspiring and actionable lessons on how to succeed as a leader in business, HR, and life. Highly recommend listening to Nine to Thrive HR if you want the knowledge AND mindsets to build your best team and grow your business! 



In [44]:
#review samples after cleaning
for i in df["cont_clean"][:3]:
    print(f"REVIEW AFTER CLEANING:", i, "\n")

REVIEW AFTER CLEANING: we listen closely case of murder and loss going west true crime stella portland oregon 

REVIEW AFTER CLEANING: love adam brooke so true the generation change but they do recycle i love the banter and found the discussion of bell bottom record player funny it brought back memory of my own discussion with my mom a a little girl i told her she should wear bell bottom because theyre so cool and she said ive been there done that and no i get it now 

REVIEW AFTER CLEANING: holly and her guest share inspiring and actionable lesson on how to succeed a a leader in business hr and life highly recommend listening to nine to thrive hr if you want the knowledge and mindset to build your best team and grow your business 



### It's hard to tell what some of the words are in the topics after stemming
### Reference for applying Lancaster and Snowball  stemmer:

In [ ]:
from nltk.stem.lancaster import LancasterStemmer

#apply Lancaster stemming
stemmer = LancasterStemmer()

start = time.time()
text_noun["content"] = text_noun["content"].apply(lambda x: [stemmer.stem(i) for i in x])
print(time.time() - start)

#joining all the nouns together
text_noun["content"] = text_noun["content"].apply(lambda x: " ".join(x))

In [ ]:
text_noun.head(3)

In [ ]:
from nltk.stem.snowball import SnowballStemmer

#apply Snowball stemming
sb_stemmer = SnowballStemmer("english")

start = time.time()
text_noun["content"] = text_noun["content"].apply(lambda x: [sb_stemmer.stem(i) for i in x])
print(time.time() - start)

#joining all the nouns together
text_noun["content"] = text_noun["content"].apply(lambda x: " ".join(x))

In [ ]:
text_noun.head(3)